In [12]:
import sys
sys.path.append('../')
sys.path.append('spsa/')


from pytorch_optim_training_manager import train_manager
import torch
import torchvision
import torchvision.transforms as transforms
import models
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from my_spsa import SPSA as SPSA

In [13]:
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False)


In [14]:
model = models.CNN_Simple()
torch.save(model.state_dict(), 'models/CNN_Simple.pt')
model.load_state_dict(torch.load('models/CNN_Simple.pt'))
loss_fn = torch.nn.CrossEntropyLoss()

In [15]:
optimizer = SPSA(model.parameters(), lr=0.001, c=0.005, alpha=0.6, gamma=0.1)

In [16]:
manager = train_manager(model, loss_fn, optimizer, training_loader, validation_loader, device=device)
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='loss')
print(torch.min(torch.tensor(losses[1])))


epoch0: train_loss: 2.2996
eval_loss: 2.2922
epoch1: train_loss: 2.2843
eval_loss: 2.2749
epoch2: train_loss: 2.2605
eval_loss: 2.2410
epoch3: train_loss: 2.2015
eval_loss: 2.1435
epoch4: train_loss: 2.0184
eval_loss: 1.8404
epoch5: train_loss: 1.5711
eval_loss: 1.3084
epoch6: train_loss: 1.1313
eval_loss: 1.0109
epoch7: train_loss: 0.9329
eval_loss: 0.8902
epoch8: train_loss: 0.8459
eval_loss: 0.8338
epoch9: train_loss: 0.7984
eval_loss: 0.7961
epoch10: train_loss: 0.7654
eval_loss: 0.7707
epoch11: train_loss: 0.7410
eval_loss: 0.7501
epoch12: train_loss: 0.7212
eval_loss: 0.7312
epoch13: train_loss: 0.7040
eval_loss: 0.7164
epoch14: train_loss: 0.6897
eval_loss: 0.7032
epoch15: train_loss: 0.6759
eval_loss: 0.6906
epoch16: train_loss: 0.6637
eval_loss: 0.6801
epoch17: train_loss: 0.6528
eval_loss: 0.6692
epoch18: train_loss: 0.6425
eval_loss: 0.6593
epoch19: train_loss: 0.6332
eval_loss: 0.6533
tensor(0.6533)


In [18]:
torch.save(torch.tensor(losses[1]), 'results/SPSA_P1.pt')